In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
from os.path import join
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

import uproot
import hist
import mplhep as hep
import seaborn as sns
from coffea import util
import numpy as np
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty

sys.path.append("../")
from azh_analysis.utils.plotting import (
    get_category_labels, get_color_list, get_category_labels
)
from azh_analysis.utils.parameters import (
    get_lumis, get_categories, 
)

year, btag = "all", "ggA"

In [ ]:
uproot.open("/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2016/SingleElectron_Run2016Bv2_preVFP/all_SingleElectron_Run2016Bv2_preVFP_file092_part_1of3_Electrons.root").keys()

In [ ]:
hep.style.use(["CMS", "fira", "firamath"])
colors = {
    "DY": "#0A9396",
    "SM-H(125)": "#E9D8A6",
    "ZZ": "#94D2BD",
    "WZ": "#9b2226",
    "tt": "#EE9B00",
    "VVV": "#bb3e03",
}
combine_dir = "/uscms/home/jdezoort/nobackup/combine/CMSSW_11_3_4/src"
masses = np.array([225, 250, 275, 300, 325, 350, 375, 400, 450, 500, 600, 700, 750, 800, 900, 1000, 1200, 1400, 1600, 1800, 2000])
limits = defaultdict(list)
for mass in masses:
    rootfile = uproot.open(
        join(combine_dir, f"higgsCombine.{year}_{btag}.AsymptoticLimits.mH"+str(mass)+".root")
    )
    try:
        limit = rootfile["limit"].arrays()["limit"]
    except: 
        continue
    print(limit)
    if len(limit)<5: continue
    limits["mass"].append(mass)
    limits["down_2s"].append(limit[0])
    limits["down_1s"].append(limit[1])
    limits["expected"].append(limit[2])
    limits["up_1s"].append(limit[3])
    limits["up_2s"].append(limit[4])

#limit_exp = abs(masses - 1100)/1000.
#limit_obs = limit_exp + 0.1*(np.random.rand()-0.5)
#limit_1s = 0.05*limit_exp
#limit_2s = 0.1*limit_exp
#x = np.arange(len(limit_exp))

fig, ax = plt.subplots(dpi=120, figsize=(8,8))
ax.fill_between(limits["mass"], limits["down_2s"], limits["up_2s"], color='#ffff01', label="95% Expected")
ax.fill_between(limits["mass"], limits["down_1s"], limits["up_1s"], color='#00ff01', label="68% Expected")
ax.plot(limits["mass"], limits["expected"], linestyle="--", color="black", label="Median Expected")
#ax.plot(limit_obs, "k-", lw=2.5, label="Observed")
ax.set_xlabel(r"$m_A$ [GeV]")
ax.set_ylabel(r"$\sigma$"+f"({btag})"+r"$\times\mathrm{BR}(A\rightarrow Zh\rightarrow ll\tau\tau)$ [fb]")
ax.set_xlim([225, 2000])
hep.cms.label("Preliminary", data=True, lumi=59.8+41.5+16.8, year="", ax=ax, fontsize=20)
plt.legend(loc="best", fontsize=15)
plt.tight_layout()
plt.show()